In [1]:
import numpy as np
import nibabel as nib
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table

In [2]:
subject_id = "100206"
subject_path = f"diffusion_data/{subject_id}/T1w/Diffusion"
dwi_img = nib.load(f'{subject_path}/data.nii.gz')
mask_img = nib.load(f'{subject_path}/nodif_brain_mask.nii.gz')

In [3]:
# Convert to numpy arrays for processing
dwi_data = dwi_img.get_fdata()
mask = mask_img.get_fdata()

In [4]:
print(f"DWI data shape: {dwi_data.shape}")  # Should be (X, Y, Z, num_volumes)
print(f"Mask shape: {mask.shape}")          # Should be (X, Y, Z)

DWI data shape: (145, 174, 145, 288)
Mask shape: (145, 174, 145)


In [5]:
# Load gradient information (bvals and bvecs)
print("\nLoading gradient information...")
bvals, bvecs = read_bvals_bvecs(f'{subject_path}/bvals', 
                               f'{subject_path}/bvecs')
print(f"Number of gradient directions: {len(bvals)}")
print(f"bvals shape: {bvals.shape}")     # Should match number of volumes
print(f"bvecs shape: {bvecs.shape}")     # Should be (num_volumes, 3)


Loading gradient information...
Number of gradient directions: 288
bvals shape: (288,)
bvecs shape: (288, 3)


In [6]:
# Create gradient table for DIPY
gtab = gradient_table(bvals, bvecs)

In [7]:
# Identify and extract B0 (non-diffusion weighted) volumes
b0_mask = gtab.b0s_mask
b0_data = dwi_data[..., b0_mask]
print(f"\nNumber of B0 volumes: {np.sum(b0_mask)}")
print(f"B0 data shape: {b0_data.shape}")


Number of B0 volumes: 18
B0 data shape: (145, 174, 145, 18)
